In [1]:
import sys
sys.path.append("../../underactuated")

In [2]:
import numpy as np
import pickle
from IPython.display import HTML, display
from utils import calc_u_opt, reconstruct_polynomial_from_dict
from quadrotor2d_sos import quadrotor2d_sos_lower_bound
from pydrake.all import (DiagramBuilder, Simulator, WrapToSystem, LeafSystem,
                         BasicVector, MathematicalProgram)
from underactuated.quadrotor2d import Quadrotor2D, Quadrotor2DVisualizer

In [12]:
class Controller(LeafSystem):
    def __init__(self, J_star, z):
        LeafSystem.__init__(self)
        quadrotor = Quadrotor2D()
        self.u0 = quadrotor.mass * quadrotor.gravity / 2. * np.array([1, 1])
        self.x_dim = 6
        self.u_dim = 2
        self.nz,  _, self.f2, self.x2z, self.Rinv = quadrotor2d_sos_lower_bound(2, test=True)
        self.dJdz = J_star.Jacobian(z)
        self.z = z
        
        self.state_input_port = self.DeclareVectorInputPort(
            "state", BasicVector(self.x_dim))
        self.policy_output_port = self.DeclareVectorOutputPort(
            "policy", BasicVector(self.u_dim), self.CalculateController)
        
    def CalculateController(self, context, output):
        x = self.state_input_port.Eval(context)
        print(x)
        z_val = self.x2z(x)
        y = output.get_mutable_value()
        f2_val = self.f2(z_val, float)
        dJdz_val = np.zeros(self.nz)
        for n in range(self.nz): 
            dJdz_val[n] = self.dJdz[n].Evaluate(dict(zip(z, z_val)))
        u_opt = calc_u_opt(dJdz_val, f2_val, self.Rinv)
        y[:]  = u_opt + self.u0  # CAUTION: Add u_equilibrium

In [48]:
def simulate(J_star, z):
    # Animate the resulting policy.
    builder = DiagramBuilder()
    plant = builder.AddSystem(Quadrotor2D())
    controller = builder.AddSystem(Controller(J_star, z))
    builder.Connect(controller.get_output_port(0), plant.get_input_port(0))
    builder.Connect(plant.get_output_port(0), controller.get_input_port(0))

    # Setup visualization
    visualizer = builder.AddSystem(Quadrotor2DVisualizer(show=False))
    builder.Connect(plant.get_output_port(0), visualizer.get_input_port(0))
    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    visualizer.start_recording()
    for i in range(1):
        context.SetTime(0.)
        context.SetContinuousState([100, 100, np.pi/2, 100, 100, 100])
        simulator.Initialize()
        simulator.AdvanceTo(10)
    print("Final state: ", context.get_continuous_state_vector().CopyToVector())
    visualizer.stop_recording()
    ani = visualizer.get_recording_as_animation()
    display(HTML(ani.to_jshtml()))

In [5]:
deg = 2
prog = MathematicalProgram()
z = prog.NewIndeterminates(7, "z")
with open("data/J_upper_bound_deg_{}.pkl".format(deg), "rb") as input_file:
    C = pickle.load(input_file)
J_star = reconstruct_polynomial_from_dict(C, z)

In [49]:
simulate(J_star, z)

Final state:  [-2.85001596e-10  6.23356554e-04 -1.88495559e+01  6.33573374e-12
 -6.74900476e-05 -2.30105993e-06]
